In [1]:
import pandas as pd
from pulp import *
from itertools import *
from mip import *
from partitions import *

Using Python-MIP package version 1.7.3


In [2]:
BUDGET = 1000000

In [3]:
def get_data():
    df = pd.read_csv('all_wines_cleaned.csv').sort_values(['Pontuação','Pontos_Total'], ascending=False)
    #Incluir somente os vinhos disponíveis e com alguma pontuação, pois essas serão nossas variáveis de decisão principais
    data = df[df.Preço_Normal.notna() & df.Pontos_Total != 0].copy().reset_index(drop=True)
    #data.set_index('Nome', inplace=True)
    return data

In [4]:
data = get_data()

In [5]:
data = data.sort_values('Pontos_Total', ascending=False)

In [6]:
data.head(10)

,Nome,Link,País,Preço_Sócio,Preço_Normal,Pontuação,Avaliações,Somelier,Decantação,Olfativo,...,Tipo,Potencial_Guarda,Região,Preços_Cat,Pontuação_Cat,Estoque_Cat,Seco,Pontos_Total,Desc_Sócio,Puro
194,Bear Flag Red Blend 2018,https://wine.com.br/vinhos/bear-flag-red-blend...,Estados Unidos,39.90,46.94,4.0,1014.0,O Bear Flag foi feito com o esforço em comum d...,NaN,"Frutas negras, com um leve tostado e um toque ...",...,Tinto,5.0,Califórnia,1.0,2,2,1,4056.0,0.149979,0
195,Pedro Teixeira Colheita Selecionada Bairrada D...,https://wine.com.br/vinhos/pedro-teixeira-colh...,Portugal,34.90,41.06,4.0,389.0,"O renomado enólogo Osvaldo Amado fez, neste vi...",NaN,"Frutas vermelhas maduras, frutas em compota, e...",...,Tinto,7.0,Bairrada,1.0,2,2,0,1556.0,0.150024,0
196,Espumante Veuve D`Argent Blanc De Blancs Brut,https://wine.com.br/vinhos/espumante-veuve-d-a...,França,36.90,43.41,4.0,360.0,"Frutas brancas como pera e maçã, delicadas not...",NaN,"Frutas brancas como pera e maçã, delicadas not...",...,Espumante,2.0,Bourgogne,1.0,2,1,0,1440.0,0.149965,0
197,Praça dos Marqueses Escolha I.G. Beira Atlânti...,https://wine.com.br/vinhos/praca-dos-marqueses...,Portugal,28.90,34.00,4.0,339.0,O nome Praça dos Marqueses é uma homenagem à p...,NaN,"Frutas vermelhas, frutas negras, floral",...,Tinto,5.0,Beira Atlântico,1.0,2,2,0,1356.0,0.150000,0
63,Clos de Los Siete By Michel Rolland 2016,https://wine.com.br/vinhos/clos-de-los-siete-b...,Argentina,78.90,92.82,4.5,268.0,Pontuado com altas notas por James Suckling e ...,20.0,"Amora, mirtilo, violeta, especiarias, amadeirado",...,Tinto,10.0,Valle de Uco,2.0,2,2,1,1206.0,0.149968,0
64,Toro Loco D.O.P. Utiel-Requena Tinto Superior ...,https://wine.com.br/vinhos/toro-loco-d-o-p-uti...,Espanha,24.90,29.29,4.5,261.0,"Mais do que um vinho, uma companhia para qualq...",NaN,"Frutas frescas, floral",...,Tinto,4.0,Utiel,1.0,2,1,0,1174.5,0.149881,0
198,Espumante Toro Loco D.O. Cava Brut,https://wine.com.br/vinhos/espumante-toro-loco...,Espanha,44.90,52.82,4.0,284.0,"Com aromas de flores brancas, frutas frescas c...",NaN,"Flores brancas, frutas frescas como maçã e per...",...,Espumante,3.0,Utiel,1.0,2,1,0,1136.0,0.149943,0
199,Viña Cosos D.O. Campo de Borja Garnacha Syrah ...,https://wine.com.br/vinhos/vina-cosos-d-o-camp...,Espanha,42.42,49.90,4.0,254.0,"Mesmo a vinícola sendo fundada em 1984, a Garn...",NaN,"Frutas vermelhas maduras, cerejas, ameixa e fl...",...,Tinto,5.0,Campo de Borja,1.0,2,2,1,1016.0,0.149900,0
65,Casillero Reserva Limited Edition Cabernet Sau...,https://wine.com.br/vinhos/casillero-reserva-l...,NaN,44.90,52.82,4.5,203.0,"Edição especial para o Halloween, esse Caberne...",NaN,"Cassis, cereja, ameixa e notas de tostado",...,Tinto,4.0,NaN,1.0,2,1,1,913.5,0.149943,1
66,Espumante Fantinel D.O.C. Prosecco Extra Dry,https://wine.com.br/vinhos/espumante-fantinel-...,Itália,54.90,64.59,4.5,187.0,"Elaborado pelo método Charmat, com ótimo fresc...",NaN,"Frutas cítricas, maçã verde, pêssego, floral",...,Espumante,3.0,Friuli,2.0,2,1,1,841.5,0.150023,1


In [7]:
vinhos = data.index.to_list()

Os vinhos do catálogo possuem cerca de 15 por cento de desconto para os associados, i.e. `Preço_Normal ~ 0.85 * Preço_Sócio`

Considerando como uma aproximação que no Preço Normal incida aproximadamente 25% do valor do Preço de Custo do Vinho, i.e. 10% do Preço_Sócio.

In [8]:
custo = data['Preço_Normal'] / 1.25

In [9]:
data['Custo'] = custo

Como devemos ter no mínimo 1000 garrafas de cada vinho escolhido, vamos modelar variáveis binárias indicando se um vinho foi escolhido para a compra. E essa compra é de 1000 vinhos.

In [13]:
custo = dict(zip(vinhos, data.Custo))

In [14]:
preço = dict(zip(vinhos, data.Preço_Normal))

In [15]:
pontos = dict(zip(vinhos, data.Pontuação))

In [16]:
n_pontos = dict(zip(vinhos, data.Avaliações.to_list()))

A coluna adicional `Total_Pontos = Pontuação * Avaliações` será usada como otimizador do vinho

In [17]:
total_pontos = dict(zip(vinhos, data.Pontos_Total.to_list()))

In [18]:
def return_solution(prob):   
    return LpStatus[prob.status], {v:v.varValue for v in prob.variables() if v.varValue > 0}  

# Solução Trivial
Maximizar a pontuação total dentro do Orçamento

In [167]:
prob = LpProblem("Maximizar somente a pontuação total", LpMaximize)

## Variável

In [18]:
vinhos_var = LpVariable.dicts("Vinhos", vinhos, lowBound=0, cat='Integer')

## Objetiva

In [19]:
prob += lpSum([total_pontos[i] * vinhos_var[i] for i in vinhos])

## Constraints
Orçamento

In [20]:
prob += lpSum([custo[i] * vinhos_var[i] for i in vinhos]) <= BUDGET

In [21]:
prob.solve()

1

In [22]:
status, solução = return_solution(prob)

In [23]:
solução

{Vinhos_194: 26629.0, Vinhos_197: 1.0}

In [24]:
solução = {int(str(k).split('Vinhos_')[1]):v for k,v in solução.items()}

In [25]:
list(solução.keys())

[194, 197]

In [26]:
solution = data.loc[solução.keys()].copy()

In [27]:
solution['Quantidade'] = solução.values()

In [28]:
solution[["Quantidade", 'Nome', 'Custo', 'Pontuação', 'Avaliações']]

,Quantidade,Nome,Custo,Pontuação,Avaliações
194,26629.0,Bear Flag Red Blend 2018,37.552,4.0,1014.0
197,1.0,Praça dos Marqueses Escolha I.G. Beira Atlânti...,27.200,4.0,339.0


A solução é trivial porque retorna o máximo possível de vinhos com a pontuação máxima, o vinho californiano `Bear Flag Red Blend` e com o valor restante compra 1 garrafa do vinho mais barato mas com boas pontuações.

Quantidade total do Estoque:

In [29]:
sum(solution.Quantidade.to_list())

26630.0

## Solução 2
Maximizar a pontuação total dentro do Orçamento mas limitar o número de garrafas à 5000.

In [30]:
prob = LpProblem("Limitar a quantidade à 5000", LpMaximize)

## Variável

In [31]:
vinhos_var = LpVariable.dicts("Vinhos", vinhos, lowBound=0, upBound=5000, cat='Integer')

## Objetiva

In [32]:
prob += lpSum([total_pontos[i] * vinhos_var[i] for i in vinhos])

## Constraints
Orçamento

In [33]:
prob += lpSum([custo[i] * vinhos_var[i] for i in vinhos]) <= BUDGET

In [34]:
prob.solve()

1

In [35]:
status, solução = return_solution(prob)

In [36]:
solução = {int(str(k).split('Vinhos_')[1]):v for k,v in solução.items()}

In [37]:
print(f'Status: {status}')
print(f'Solução Encontrada: {solução}')

Status: Optimal
Solução Encontrada: {194: 5000.0, 195: 5000.0, 196: 5000.0, 197: 5000.0, 198: 4999.0, 199: 247.0, 200: 4.0, 64: 5000.0}


In [38]:
solution = data.loc[solução.keys()].copy()

In [39]:
solution['Quantidade'] = solução.values()

In [40]:
solution[["Quantidade", 'Nome', 'Custo', 'Pontuação', 'Avaliações']].sort_values("Quantidade",ascending=False)

,Quantidade,Nome,Custo,Pontuação,Avaliações
194,5000.0,Bear Flag Red Blend 2018,37.552,4.0,1014.0
195,5000.0,Pedro Teixeira Colheita Selecionada Bairrada D...,32.848,4.0,389.0
196,5000.0,Espumante Veuve D`Argent Blanc De Blancs Brut,34.728,4.0,360.0
197,5000.0,Praça dos Marqueses Escolha I.G. Beira Atlânti...,27.200,4.0,339.0
64,5000.0,Toro Loco D.O.P. Utiel-Requena Tinto Superior ...,23.432,4.5,261.0
198,4999.0,Espumante Toro Loco D.O. Cava Brut,42.256,4.0,284.0
199,247.0,Viña Cosos D.O. Campo de Borja Garnacha Syrah ...,39.920,4.0,254.0
200,4.0,Baron Philippe de Rothschild Mas Andes Caberne...,25.320,4.0,157.0


Quantidade de Garrafas no Estoque

In [41]:
sum(solution.Quantidade.to_list())

30250.0

A solução ainda é trivial porque há um catálogo muito pobre, seis vinhos somente com grande estoque e outros 2 com estoque baixo. No entanto a quantidade de garrafas é superior ao primeiro caso. Nada mais que ordenar pela quantidade de avaliações total.

## Solução 3
Maximizar a Pontuação total e com pontuação mínima 4

In [42]:
prob = LpProblem("Rastrear a Pontuação Mínima", LpMaximize)

## Variável
Número de Garrafas Limitada à 5000 para evitar a solução trivial

In [43]:
vinhos_var = LpVariable.dicts("Vinhos", vinhos, lowBound=0, upBound=5000, cat='Integer')

## Objetiva
Maximizar a pontuação total

In [44]:
prob += lpSum([total_pontos[i] * vinhos_var[i] for i in vinhos])

## Constraints
Orçamento

In [45]:
prob += lpSum([custo[i] * vinhos_var[i] for i in vinhos]) <= BUDGET

Pontuação Mínima deve ser 4

In [46]:
prob += lpSum([pontos[i] * vinhos_var[i] for i in vinhos]) >= sum([4 * vinhos_var[i] for i in vinhos]) 

In [47]:
prob.solve()

1

In [48]:
status, solução = return_solution(prob)

In [49]:
solução = {int(str(k).split('Vinhos_')[1]):v for k,v in solução.items()}

In [50]:
print(f'Status: {status}')
print(f'Solução Encontrada: {solução}')

Status: Optimal
Solução Encontrada: {194: 5000.0, 195: 5000.0, 196: 5000.0, 197: 5000.0, 198: 4999.0, 199: 247.0, 200: 4.0, 64: 5000.0}


In [51]:
solution = data.loc[solução.keys()].copy()

In [52]:
solution['Quantidade'] = solução.values()

In [53]:
solution[["Quantidade", 'Nome', 'Custo', 'Pontuação', 'Avaliações']].sort_values("Quantidade",ascending=False)

,Quantidade,Nome,Custo,Pontuação,Avaliações
194,5000.0,Bear Flag Red Blend 2018,37.552,4.0,1014.0
195,5000.0,Pedro Teixeira Colheita Selecionada Bairrada D...,32.848,4.0,389.0
196,5000.0,Espumante Veuve D`Argent Blanc De Blancs Brut,34.728,4.0,360.0
197,5000.0,Praça dos Marqueses Escolha I.G. Beira Atlânti...,27.200,4.0,339.0
64,5000.0,Toro Loco D.O.P. Utiel-Requena Tinto Superior ...,23.432,4.5,261.0
198,4999.0,Espumante Toro Loco D.O. Cava Brut,42.256,4.0,284.0
199,247.0,Viña Cosos D.O. Campo de Borja Garnacha Syrah ...,39.920,4.0,254.0
200,4.0,Baron Philippe de Rothschild Mas Andes Caberne...,25.320,4.0,157.0


A solução encontrada é idêntica à solução 2 tendo em vista que já obedecia ao Constraint adicional.

## Solução 4

In [83]:
prob = LpProblem("O preço como variável objetiva", LpMaximize)

## Variável
Número de Garrafas Limitada à 5000 para evitar a solução trivial

In [84]:
vinhos_var = LpVariable.dicts("Vinhos", vinhos, lowBound=0, upBound=5000, cat='Integer')

## Objetiva
Maximizar a pontuação total

In [85]:
prob += lpSum([vinhos_var[i] * 1./preço[i] for i in vinhos])

## Constraints
Orçamento

In [86]:
prob += lpSum([custo[i] * vinhos_var[i] for i in vinhos]) <= BUDGET

Para a Pontuação Mínima ser 4, devemos majorar a pontuação média por um valor maior

In [96]:
prob += lpSum([pontos[i] * vinhos_var[i] for i in vinhos]) >= sum([4.5 * vinhos_var[i] for i in vinhos])  

In [97]:
prob.solve()

1

In [98]:
status, solução = return_solution(prob)

In [99]:
solução = {int(str(k).split('Vinhos_')[1]):v for k,v in solução.items()}

In [100]:
print(f'Status: {status}')
print(f'Solução Encontrada: {solução}')

Status: Optimal
Solução Encontrada: {121: 5000.0, 151: 5000.0, 153: 3404.0, 160: 5000.0, 216: 5000.0, 295: 5000.0, 56: 5000.0, 62: 5000.0, 64: 5000.0}


In [101]:
solution = data.loc[solução.keys()].copy()

In [102]:
solution['Quantidade'] = solução.values()

In [103]:
solution[["Quantidade", 'Nome', 'Custo', 'Pontuação', 'Avaliações']].sort_values("Quantidade",ascending=False)

,Quantidade,Nome,Custo,Pontuação,Avaliações
121,5000.0,Dark Horse Rosé Bubbles Lata 375ml,23.432,4.5,14.0
151,5000.0,Dark Horse Pinot Grigio Lata 375ml,23.432,4.5,8.0
160,5000.0,Dark Horse Rosé Lata 375ml,23.432,4.5,6.0
216,5000.0,Terralis Syrah Malbec 2018,20.608,4.0,55.0
295,5000.0,Espumante Baby Chandon Réserve Brut 187 ml.,22.496,4.0,14.0
56,5000.0,Frisante Miolo Moscatel Blanc,22.496,5.0,1.0
62,5000.0,Viña Carrasco Rosé 2018,23.432,5.0,1.0
64,5000.0,Toro Loco D.O.P. Utiel-Requena Tinto Superior ...,23.432,4.5,261.0
153,3404.0,Canepa Novísimo Carménère 2019,25.320,4.5,7.0


In [7]:
#data.index = data.index.str.replace(" ", "_")

Quantidade de Garrafas no Estoque

In [105]:
sum(solution.Quantidade.to_list())

43404.0

Está claro que se não embutirmos constraints mais sofisticados somente temos soluções triviais, encontradas facilmente simplesmente ordenando  DataFrame

## Solução 4
* Minimizar o Custo como variável objetiva
* Maximixar a Pontuação
* Maximizar o nº de Avaliações

In [250]:
prob = LpProblem("Maximo Pontuação, mínimo Preço", LpMaximize)

/home/ronaldo/.local/share/virtualenvs/wino-rEbptTiD/lib/python3.7/site-packages/pulp/pulp.py:1137: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


## Variável
Número de Garrafas Limitada à 5000 para evitar a solução trivial

In [251]:
vinhos_var = LpVariable.dicts("Vinhos", vinhos, lowBound=0, upBound=5000, cat='Integer')

## Objetiva
Maximizar a pontuação total

In [252]:
prob += lpSum([vinhos_var[i] * 1./preço[i] for i in vinhos])

## Constraints
Orçamento

In [253]:
prob += lpSum([custo[i] * vinhos_var[i] for i in vinhos]) <= BUDGET

Pontuação Mínima deve ser 4

In [254]:
prob += lpSum([pontos[i] * vinhos_var[i] for i in vinhos]) >= 4.5 * sum([vinhos_var[i] for i in vinhos]) 

In [255]:
prob.solve()

1

In [256]:
status, solução = return_solution(prob)

In [257]:
solução = {int(str(k).split('Vinhos_')[1]):v for k,v in solução.items()}

In [258]:
print(f'Status: {status}')
print(f'Solução Encontrada: {solução}')

Status: Optimal
Solução Encontrada: {241: 2.0, 260: 5000.0, 280: 5000.0, 345: 5000.0, 471: 2350.0, 500: 5000.0, 502: 5000.0, 503: 2350.0, 62: 5000.0}


In [259]:
solution = data.loc[solução.keys()].copy()

In [260]:
solution['Quantidade'] = solução.values()

In [261]:
solution[["Quantidade", 'Nome', 'Custo', 'Pontuação', 'Avaliações']].sort_values("Quantidade",ascending=False)

,Quantidade,Nome,Custo,Pontuação,Avaliações
260,5000.0,Nederburg 56 Hundred Pinot Grigio 2018,25.320,4.0,25.0
280,5000.0,Mosaiko 2015,37.552,4.0,18.0
345,5000.0,Dark Horse Brut Bubbles Lata 375ml,23.432,4.0,7.0
500,5000.0,Viña Pelequen Cabernet Sauvignon/Carmenere Syr...,26.256,3.5,2.0
502,5000.0,Viña Carrasco Chardonnay 2018,23.432,3.5,2.0
62,5000.0,Viña Carrasco Rosé 2018,23.432,5.0,1.0
471,2350.0,Dark Horse Sauvignon Blanc Lata 375ml,23.432,3.5,6.0
503,2350.0,Zenithwirl Cabernet Sauvignon 2018,30.024,3.0,46.0
241,2.0,Canepa Novísimo Sauvignon Blanc 2018,35.680,4.0,34.0


Quantidade de Garrafas no Estoque

In [262]:
sum(solution.Quantidade.to_list())

34702.0

In [265]:
sum(row.Quantidade * row.Custo for row in solution.itertuples()) <= BUDGET

True

Está claro que se não embutirmos constraints mais sofisticados somente temos soluções triviais, encontradas facilmente simplesmente ordenando  DataFrame

## Estratégia
Os dados de negócio da wine.com são:
* 100.000 clientes
* 2 vinhos distintos por mês
* 569 vinhos disponíveis
* 3780 vinhos distintos no catálogo

Mock-up de Negócios: 1% do escopo da wine com Orçamento de R$ 1.000.000
* Projeção de 1000 Clientes
* 2 vinhos por mês
* Estoque para 6 meses

Isso resulta inicialmente no mínimo de 12000 garrafas no estoque, 12 vinhos distintos e no mínimo de 1000 garrafas por vinho. 
Vamos começar com essas condições iniciais.

In [10]:
data.set_index('Nome', inplace=True)

In [11]:
vinhos = data.index.to_list()
n = range(len(vinhos))
custo = [float(i) for i in data.Custo.to_list()]
pontos = [float(i) for i in data.Pontuação.to_list()]
avaliação = [int(i) for i in data.Avaliações.to_list()]
total_pontos = [int(i) for i in data.Pontos_Total.to_list()]
NUM_WINE = 1000

In [12]:
m = Model('Competidor') # use GRB for Gurobi

In [13]:
wines = [m.add_var(name=vinho, var_type=BINARY) for vinho in vinhos]

In [14]:
m.objective = maximize(xsum(wines[i] * total_pontos[i] for i in n))

In [15]:
m += xsum(wines[i] * custo[i] for i in n) <= BUDGET / NUM_WINE

In [16]:
m += xsum(wines[i] * avaliação[i] >= 4 for i in n) >= xsum(1 * wines[i] for i in n)

Relaxação básica do orçamento, se tivermos um valor menor no orçamento do que o vinho mais barato não podemos comprar a safra

In [398]:
#m += xsum(wines[i] * custo[i] for i in n) >= BUDGET / NUM_WINE - data.Custo.min()

In [17]:
m.max_gap = 0.05
status = m.optimize(max_seconds=300)
if status == OptimizationStatus.OPTIMAL:
    print('optimal solution cost {} found'.format(m.objective_value))
elif status == OptimizationStatus.FEASIBLE:
    print('sol.cost {} found, best possible: {}'.format(m.objective_value, m.objective_bound))
elif status == OptimizationStatus.NO_SOLUTION_FOUND:
    print('no feasible solution found, lower bound is: {}'.format(m.objective_bound))
if status == OptimizationStatus.OPTIMAL or status == OptimizationStatus.FEASIBLE:
    print('solution:')
    for v in m.vars:
        if abs(v.x) > 1e-6: # only printing non-zeros
            print('{} : {}'.format(v.name, v.x))

optimal solution cost 21188.0 found
solution:
Bear Flag Red Blend 2018 : 1
Pedro Teixeira Colheita Selecionada Bairrada D.O.C 2018 : 1
Espumante Veuve D`Argent Blanc De Blancs Brut : 1
Praça dos Marqueses Escolha I.G. Beira Atlântico 2018 : 1
Clos de Los Siete By Michel Rolland 2016 : 1
Toro Loco D.O.P. Utiel-Requena Tinto Superior 2018 : 1
Espumante Toro Loco D.O. Cava Brut : 1
Viña Cosos D.O. Campo de Borja Garnacha Syrah 2018 : 1
Casillero Reserva Limited Edition Cabernet Sauvignon : 1
Espumante Fantinel D.O.C. Prosecco Extra Dry : 1
Baron Philippe de Rothschild Mas Andes Cabernet Sauvignon 2018 : 1
Baron Philippe de Rothschild Mas Andes Carménère 2018 : 1
Espumante Louis Bouillot Rosé Brut : 1
Sardelli D.O.C.G. Chianti 2016 : 1
Tenuta Sant`Antonio Scaia I.G.T. Veneto Corvina 2015 : 1
Roureda Levante D.O. Tarragona 2017 : 1
Pérez Cruz GSM D.O. Valle del Maipo Andes 2016 : 1
Dancing Flame Ojos del Salado Shiraz 2018 : 1
Espumante Salton Moscatel : 1
Canepa Novísimo Merlot 2018 : 1
Te

In [18]:
solução = {v.name:v.x for v in m.vars if abs(v.x) > 1e-6}

In [19]:
solution = data.loc[solução.keys()].copy()

In [26]:
solution[['Custo', 'Preço_Normal', 'Preço_Sócio', 'Pontuação', 'Avaliações', 'Pontos_Total', 'Potencial_Guarda']].sort_values("Potencial_Guarda")

,Custo,Preço_Normal,Preço_Sócio,Pontuação,Avaliações,Pontos_Total,Potencial_Guarda
Nome,,,,,,,
Espumante Veuve D`Argent Blanc De Blancs Brut,34.728,43.41,36.90,4.0,360.0,1440.0,2.0
Espumante Salton Moscatel,30.968,38.71,32.90,4.5,72.0,324.0,2.0
Canepa Novísimo Merlot 2018,25.320,31.65,26.90,4.5,72.0,324.0,3.0
Espumante Salton Brut,30.968,38.71,32.90,4.5,60.0,270.0,3.0
Espumante Toro Loco D.O. Cava Brut,42.256,52.82,44.90,4.0,284.0,1136.0,3.0
Canepa Novísimo Syrah 2018,25.320,31.65,26.90,4.0,53.0,212.0,3.0
Espumante Fantinel D.O.C. Prosecco Extra Dry,51.672,64.59,54.90,4.5,187.0,841.5,3.0
Espumante Louis Bouillot Rosé Brut,61.080,76.35,64.90,4.5,109.0,490.5,3.0
Baron Philippe de Rothschild Mas Andes Cabernet Sauvignon 2018,25.320,31.65,26.90,4.0,157.0,628.0,4.0


In [23]:
solution.shape

(29, 31)

In [22]:
sum(row.Quantidade * row.Custo for row in solution.itertuples()) * 1000 < BUDGET

True

In [28]:
p = [10, 13, 18, 31, 7, 15]
w = [11, 15, 20, 35, 10, 33]
c, I = 47, range(len(w))

m = Model('knapsack')

x = [m.add_var(var_type=BINARY) for i in I]

m.objective = maximize(xsum(p[i] * x[i] for i in I))

m += xsum(w[i] * x[i] for i in I) <= c

m.optimize()

selected = [i for i in I if x[i].x >= 0.99]
print('selected items: {}'.format(selected))

selected items: [0, 3]


In [ ]:
prob = LpProblem("Maximização_do_Catálogo", LpMinimize)

In [ ]:
vinho_vars = LpVariable.dicts("Vinhos", vinhos, lowBound=0, cat='Continous')

In [ ]:
prob += lpSum([total_pontos[i]* vinho_vars[i] for i in vinho_vars])

Orçamento

In [ ]:
prob += lpSum([custo[i] * vinho_vars[i] for i in vinho_vars]) <= BUDGET

In [ ]:
prob.solve()

In [ ]:
for v in prob.variables():
    print(v.name, "=", v.varValue)

In [ ]:
pontos.sum()

In [ ]:
n_pontos.sum()

Primeiramente vamos ver o que temos só minimizando o custo dentro do Orçamento.

In [ ]:
vinhos_possiveis = [tuple(c) for c in allcombinations(vinhos, len(vinhos))]

In [ ]:
import pulp

max_tables = 5
max_table_size = 4
guests = 'A B C D E F G I J K L M N O P Q R'.split()

def happiness(table):
    """
    Find the happiness of the table
    - by calculating the maximum distance between the letters
    """
    return abs(ord(table[0]) - ord(table[-1]))
                
#create list of all possible tables
possible_tables = [tuple(c) for c in pulp.allcombinations(guests, 
                                        max_table_size)]

#create a binary variable to state that a table setting is used
x = pulp.LpVariable.dicts('table', possible_tables, 
                            lowBound = 0,
                            upBound = 1,
                            cat = pulp.LpInteger)

seating_model = pulp.LpProblem("Wedding Seating Model", pulp.LpMinimize)

seating_model += sum([happiness(table) * x[table] for table in possible_tables])

#specify the maximum number of tables
seating_model += sum([x[table] for table in possible_tables]) <= max_tables, \
                            "Maximum_number_of_tables"

#A guest must seated at one and only one table
for guest in guests:
    seating_model += sum([x[table] for table in possible_tables
                                if guest in table]) == 1, "Must_seat_%s"%guest

seating_model.solve()

print("The choosen tables are out of a total of %s:"%len(possible_tables))
for table in possible_tables:
    if x[table].value() == 1.0:
        print(table)

In [ ]:
ord('M') - ord('N')

In [ ]:
from partitions import get_partitions_ruskey as getpart

In [ ]:
for i, p in enumerate(getpart(data.index.values)):
    if i == 1000:
        break
    print(len(p))